* import necessary modules

In [1]:
# Python standard modules
import os, sys
import glob

# scientific modules
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import cv2 # opencv is little bit faster than pillow

# utilities
from tqdm import tqdm

# my custom modules
import download_extract as doex # it works don't care about ide.


First things first you should download the data from url: https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
You can download manually or use functions below.


In [ ]:
challenge_file = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'            
chunk=1024

doex.download_extract(challenge_file, chunk)


please don't interrupt
file lenght is 805561.08203125kb


196727it [00:45, 3321.02it/s]

Load the files into the Python 


In [ ]:
DATADIR = os.getcwd()
CATEGORIES = ["Dog", "Cat"]

paths = [[],[]] # get all the paths
for i,categories in enumerate(CATEGORIES): # for all categories
    path = os.path.join(DATADIR+"\\PetImages", categories) # create path
    for img_path in os.scandir(path):
        paths[i].append(os.path.join(path,img_path))


In [ ]:
# inspect and adjust some configurations for sample image
# select a random sample
import secrets
secure_random = secrets.SystemRandom()
sample_img_path = secure_random.choice(secure_random.choice(paths)) # 2d list
img_array = cv2.imread(sample_img_path, cv2.IMREAD_GRAYSCALE) #single channel image
print(f"shape: {img_array.shape}")
print(f"array: {img_array}")

plt.figure()
plt.imshow(img_array, cmap='gray')
plt.title("original")
plt.show()

# image is too big for networks.
# need to resize
IMG_SIZE=50 # blurry image :(
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.figure()
plt.imshow(new_array, cmap="gray")
plt.title(f"resized to {IMG_SIZE}")
plt.show()

# little bit higher image size
IMG_SIZE=100
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.figure()
plt.imshow(new_array, cmap="gray")
plt.title(f"resized to {IMG_SIZE}")
plt.show()


prepare data for training
* gray images = (N, IMG_SIZE, IMG_SIZE)
* colored images = (N, IMG_SIZE, IMG_SIZE)

In [ ]:
# select them equal number of all categories
from functools import reduce # Python3
from operator import add
secure_random.shuffle(paths[0])
secure_random.shuffle(paths[1])
#def flatten(input): # 2d array flatten
#    return [j for sub in input for j in sub]

# TODO!: Fix it to experiment with faster method.
#paths_all = flatten(paths) # now it is 1d list
# experimental method. TODO! i==187 error. image is shape under IMG_SIZE 
#N = len(CATEGORIES) * len(paths[0]) # paths[0] == paths[1]
#traing_data = np.empty((N, IMG_SIZE, IMG_SIZE), dtype=np.uint8) # preallocation makes more faster
#for i, fpath in enumerate(paths_all):
#    img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
#    traing_data[i, ...] = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
#    break
#print(traing_data)

training_data = []
def create_training_data():
    for class_num, paths_category in enumerate(paths):  # do dogs and cats

        #path = os.path.join(DATADIR+"\\PetImages",category)  # create path to dogs and cats
        #class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat
        
        #print(f"{paths_category}:")
        for img in tqdm(paths_category):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(img ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()
print(f"len(training_data): {len(training_data)}")
secure_random.shuffle(training_data)

# get samples
for i,sample in enumerate(training_data[:10]):
    print(f"{i} -> {sample[1]}")


In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()
print('Saving is done')


In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)
print('Loading is done')


it is time to design a neural network
import tensorflow and keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# load the labeled data
pickle_in = open('X.pickle','rb')
X = pickle.load(pickle_in)
pickle_in = open('y.pickle','rb')
y = pickle.load(pickle_in)
print('Loading is done')

# Normalize the input
X = X/255.0

model = Sequential()
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))


In [ ]:
# compile and train the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3)

Observe and tune model with tensorboard

In [ ]:
import time
#from tensorflow.keras.callbacks import TensorBoard   #tensorflow2.0.0 evolving rapidly

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
# if you want to see training process, train model again.
# but no need to construct again

NAME = f"Cats-vs-dogs-CNN-{int(time.time())}"
# callback function
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f"logs\{NAME}")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# just add callback function
# to see how our model is evolving while epochs increase
model.fit(X, y,
          batch_size=32,
          epochs=10,
          validation_split=0.3,
          callbacks=[tensorboard])

lets change network a little bit and try again.

In [ ]:
model = Sequential()
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

#model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

NAME = f"Cats-vs-dogs-CNN-NODENSE64-{int(time.time())}"
# callback function
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f"logs\{NAME}")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# just add callback function
# to see how our model is evolving while epochs increase
model.fit(X, y,
          batch_size=32,
          epochs=20,
          validation_split=0.3,
          callbacks=[tensorboard])
# very bad results after 5th epoch